### Setup

In [ ]:
import os, sys
sys.path.append('..')

from cobra import io
import geckopy as gp

# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/fill/xmls/MNL_iCre1355_auto_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
altered_dir = '../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):

    # Exclude the /h directory from search
    if root.endswith('/h'):
        continue

    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

### Uniprot Gene IDs

In [ ]:
"""
The following reactions need to have gene associations updated to UniProt IDs for GECKO Kcat values:
    - ALT_MVAS, ALT_MVAD, ALT_MVAE, ALT_MVD, ALT_MVK, ALT_PMK, ALT_IDLI (MVA pathway)
    - SS, SMO, ERG5, ERG3, ERG (Sterol pathway)
    - ALT_PSPPS2, ALT_SQS2, ALT_SQE2 (Overexpressions)
    - DXS, CMK, CMS, HDS (MEP pathway)
    - Additional SQS, SQE strains with their kcat values

Concepts to keep in mind:
    - One gene may regulate multiple reactions
    - One reaction may be regulated by the conjunction or disjunction of multiple genes
"""
reac_gene = {
    # MVA Pathway -- Note: what strain are we using for MVA?
    'ALT_MVAS': ['MVAS'],
    'ALT_MVAD': ['MVAD'],
    'ALT_MVAE': ['MVAE'],
    'ALT_MVK': ['MVK'],
    'ALT_PMK': ['PMK'],
    'ALT_IDLI': ['IDLI'],
    # MEP Pathway
    'DXS': ['Cre07.g356350.t1.1'],
    'CMK': ['Cre02.g145050.t1.2'],
    'CMS': ['Cre16.g679669.t1.1'],
    'HDS': ['Cre12.g490350.t1.1'],
    # Sterol Pathway
    'SS': ['Cre03.g175250.t1.2', 'Cre03.g175250.t2.1'],
    'SMO': ['Cre17.g734644.t1.1'],
    'ERG5': ['ERG5'],
    'ERG3': ['ERG3'],
    'ERG': ['ERG4'],
    # Overexpressions
    'ALT_PSPPS2': ['SQS2'],
    'ALT_SQS2': ['SQS2'],
    'ALT_SQE2': ['SQE2'],
}

gene_uniprot = {
    # MVA Pathway
    'MVAS': 'Q9FD71', # https://www.uniprot.org/uniprotkb/Q9FD71/entry
    'MVAE': 'Q9FD65', # https://www.uniprot.org/uniprotkb/Q9FD65/publications
    'MVK': '',
    'PMK': '',
    'MVAD': '',
    'IDLI': '',
    # MEP Pathway
    'Cre07.g356350.t1.1': 'O81954', # https://www.uniprot.org/uniprotkb/O81954/entry
    'Cre02.g145050.t1.2': 'O81014', # https://www.uniprot.org/uniprotkb/O81014/entry (assumes CMK/ISPE of Arabidopsis thaliana)
    'Cre16.g679669.t1.1': '',
    'Cre12.g490350.t1.1': '',
    # Sterol Pathway
    'Cre03.g175250.t1.2': 'B4DWP0', # https://www.uniprot.org/uniprotkb/B4DWP0/entry (assumes sqs of homo sapiens)
    'Cre03.g175250.t2.1': 'B4DWP0', # ... (likewise)
    'Cre17.g734644.t1.1': 'P52020', # https://www.uniprot.org/uniprotkb/P52020/publications (assumes sqe of rattus norvegicus)
    # 'ERG3': '',
    # 'ERG5': '',
    # 'ERG4': '',
    # Overexpressions (these will be storing an object with different gene id for strains)
    'SQS2': {},
    'SQE2': {},
}

The following script rebuilds the model gene associations where the above stated gene ids are replaced with the uniprot equivalent:

### Build GECKO Model

### Gap-fill Missing Kcat

### FBA